# 使用ONNX导出PyTorch模型

本节我们将使用ONNX导出PyTorch模型，并在TensorFlow中运行该模型

我们已经从课程中了解在PyTorch中ONNX采用Tracing的方式记录模型，因此我们需要创建一个“伪输入”。然后，我们通过调用torch.onnx.export()方法生成.onnx文件。该文件以一种通用的中间表示记录了PyTorch模型，从而能够应用在不同的开发环境中。

随后，我们通过使用onnx_tf库提供prepare()方法将ONNX模型转换为Tensorflow模型并执行推理。 



In [ ]:
pip install onnx onnx_tf

In [ ]:
# Download Pretrained Model
!gdown --id 1omW_gpSpedWjo5GahxzksNW-J8MGrfdk --output convnet.pth

# Download Testing Image
!gdown --id 1A47zfadA6oXGhMvI-iCRj6CWB7rS8dWz --output digit_image.jpg

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
from PIL import Image

import tensorflow as tf

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op
    
model = ConvNet()

In [ ]:
PATH_TO_MODEL = "./convnet.pth"
model.load_state_dict(torch.load(PATH_TO_MODEL, map_location="cpu"))

In [ ]:
image = Image.open("./digit_image.jpg")

In [ ]:
def image_to_tensor(image):
    gray_image = transforms.functional.to_grayscale(image)
    resized_image = transforms.functional.resize(gray_image, (28, 28))
    input_image_tensor = transforms.functional.to_tensor(resized_image)
    input_image_tensor_norm = transforms.functional.normalize(input_image_tensor, (0.1302,), (0.3069,))
    return input_image_tensor_norm

In [ ]:
input_tensor = image_to_tensor(image)

In [ ]:
model.eval()
for p in model.parameters():
    p.requires_grad_(False)

In [ ]:
demo_input = torch.ones(1, 1, 28, 28)
torch.onnx.export(model, demo_input, "convnet.onnx")

将ONNX模型转换为Tensorflow模型并在Tensorflow环境中执行推理

In [24]:
import onnx
from onnx_tf.backend import prepare

model_onnx = onnx.load("./convnet.onnx")
tf_rep = prepare(model_onnx)
output = tf_rep.run(input_tensor.unsqueeze(0))
print(output)
# tf_rep.export_graph("./convnet.pb")

Outputs(_0=array([[-9.3505077e+00, -1.2089332e+01, -2.2392217e-03, -8.9247789e+00,
        -9.8197231e+00, -1.3349855e+01, -9.0459862e+00, -1.4492420e+01,
        -6.3023319e+00, -1.2282766e+01]], dtype=float32))
